### Data Cleaning

1. Loading the file using pandas library
2. Cleaning the dataset

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [49]:
df = pd.read_csv("data_scraped.csv")
df.head(5)

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),-1
1,1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1
2,2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,Company - Private,Security Services,Business Services,$100 to $500 million (USD),-1
3,3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa..."
4,4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"


In [3]:
#Drop unnamed columns
df= df.drop(["Unnamed: 0"],axis=1)

In [5]:
df["Salary Estimate"] = df["Salary Estimate"].replace("-1",np.nan)#remove -1 values
df = df.dropna()
df = df.reset_index() #reset the index

In [11]:
#separate the hourly salary and employer provided salary from the Salary estimate column
df["Hourly"]= df["Salary Estimate"].apply(lambda x: 1 if "per hour" in x.lower() else 0)
df["Employer provided"]= df["Salary Estimate"].apply(lambda x: 1 if "employer provided" in x.lower() else 0)
salary = df["Salary Estimate"].apply(lambda x: x.split("(")[0])
salary

0       $53K-$91K 
1      $63K-$112K 
2       $80K-$90K 
3       $56K-$97K 
4      $86K-$143K 
          ...     
737    $58K-$111K 
738    $72K-$133K 
739     $56K-$91K 
740    $95K-$160K 
741    $61K-$126K 
Name: Salary Estimate, Length: 742, dtype: object

In [12]:
#Removing the K, $ sign, per hour and employer provided salary strings
salary = salary.str.replace("K","")
salary = salary.str.replace("$","")

salary = salary.apply(lambda x: x.lower().replace("per hour",""))
salary = salary.apply(lambda x: x.lower().replace("employer provided salary:",""))
salary

<ipython-input-12-0f9a0d3eea79>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  salary = salary.str.replace("$","")


0       53-91 
1      63-112 
2       80-90 
3       56-97 
4      86-143 
        ...   
737    58-111 
738    72-133 
739     56-91 
740    95-160 
741    61-126 
Name: Salary Estimate, Length: 742, dtype: object

In [13]:
#Split salary into lower and upper
sal = salary.str.split("-",1,expand=True).rename(columns=({0:"Lower",1:"Upper"}))
sal

,Lower,Upper
0,53,91
1,63,112
2,80,90
3,56,97
4,86,143
...,...,...
737,58,111
738,72,133
739,56,91
740,95,160


In [14]:
sal.Lower = pd.to_numeric(sal.Lower)
sal.Upper = pd.to_numeric(sal.Upper)
df["Lower Salary"] = sal.Lower
df["Upper Salary"] = sal.Upper

In [15]:
df['Lower Salary']= df.apply(lambda x: x['Lower Salary'] if x['Hourly']==0 else (int)(x['Lower Salary']*40*52/1000), axis = 1)
df['Upper Salary']= df.apply(lambda x: x['Upper Salary'] if x['Hourly']==0 else (int)(x['Upper Salary']*40*52/1000), axis = 1)

In [16]:
df["Avg Salary(K)"] = (df["Lower Salary"]+df["Upper Salary"])/2

In [17]:
df.head(5)

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Hourly,Employer provided,Lower Salary,Upper Salary,Avg Salary(K)
0,0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),-1,0,0,53,91,72.0
1,1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1,0,0,63,112,87.5
2,2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,Company - Private,Security Services,Business Services,$100 to $500 million (USD),-1,0,0,80,90,85.0
3,3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa...",0,0,56,97,76.5
4,4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee",0,0,86,143,114.5


In [52]:
# We need to transform the Founded column into the age of the company.
df["Age"] = df["Founded"].apply(lambda x: x if x<1 else 2022 - x)
#Age of the company as of now (2022)

In [53]:
df["Age"].head(3)

0    49
1    38
2    12
Name: Age, dtype: int64

In [20]:
#Extracting skills from job desc

#Python
df["Python"] = df["Job Description"].apply(lambda x: 1 if "python" in x.lower() else 0)
df.Python.value_counts()

1    392
0    350
Name: Python, dtype: int64

In [21]:
# R
df["R-studio"] = df["Job Description"].apply(lambda x: 1 if "r-studio" in x.lower() or "r studio" in x.lower() else 0)
df["R-studio"].value_counts()

0    740
1      2
Name: R-studio, dtype: int64

In [22]:
#spark
df['spark']  = df['Job Description'].apply(lambda x: 1 if 'spark' in x.lower() else 0)
df.spark.value_counts()

0    575
1    167
Name: spark, dtype: int64

In [23]:
#aws
df['aws']  = df['Job Description'].apply(lambda x: 1 if 'aws' in x.lower() else 0)
df.aws.value_counts()

0    566
1    176
Name: aws, dtype: int64

In [24]:
#sql
df['sql']  = df['Job Description'].apply(lambda x: 1 if 'sql' in x.lower() else 0)
df.sql.value_counts()

1    380
0    362
Name: sql, dtype: int64

In [25]:
#sas 
df['sas']  = df['Job Description'].apply(lambda x: 1 if (' sas ' in x.lower() or ' sas,' in x.lower() or ' sas.' in x.lower())  else 0)
df.sas.value_counts()

0    676
1     66
Name: sas, dtype: int64

In [26]:
#excel
df['excel']  = df['Job Description'].apply(lambda x: 1 if 'excel' in x.lower() else 0)
df.excel.value_counts()

1    388
0    354
Name: excel, dtype: int64

In [27]:
#julia
df['julia']  = df['Job Description'].apply(lambda x: 1 if 'julia' in x.lower() else 0)
df.julia.value_counts()

0    742
Name: julia, dtype: int64

In [28]:
#keras
df['keras']  = df['Job Description'].apply(lambda x: 1 if 'keras' in x.lower() else 0)
df.keras.value_counts()

0    713
1     29
Name: keras, dtype: int64

In [29]:
#pytorch
df['pytorch']  = df['Job Description'].apply(lambda x: 1 if 'pytorch' in x.lower() else 0)
df.pytorch.value_counts()

0    703
1     39
Name: pytorch, dtype: int64

In [30]:
#scikit-learn
df['scikit']  = df['Job Description'].apply(lambda x: 1 if 'scikit' in x.lower() else 0)
df.scikit.value_counts()

0    688
1     54
Name: scikit, dtype: int64

In [31]:
#tensor
df['tensor']  = df['Job Description'].apply(lambda x: 1 if 'tensor' in x.lower() else 0)
df.tensor.value_counts()

0    670
1     72
Name: tensor, dtype: int64

In [32]:
#selenium
df['selenium']  = df['Job Description'].apply(lambda x: 1 if 'selenium' in x.lower() else 0)
df.selenium.value_counts()

0    739
1      3
Name: selenium, dtype: int64

In [33]:
#hadoop
df['hadoop']  = df['Job Description'].apply(lambda x: 1 if 'hadoop' in x.lower() else 0)
df.hadoop.value_counts()

0    618
1    124
Name: hadoop, dtype: int64

In [34]:
#tableau
df['tableau']  = df['Job Description'].apply(lambda x: 1 if 'tableau' in x.lower() else 0)
df.tableau.value_counts()

0    594
1    148
Name: tableau, dtype: int64

In [35]:
#power bi
df['bi']  = df['Job Description'].apply(lambda x: 1 if ('power bi' in x.lower() or 'powerbi' in x.lower()) else 0)
df.bi.value_counts()

0    686
1     56
Name: bi, dtype: int64

In [36]:
#apache flink
df['flink']  = df['Job Description'].apply(lambda x: 1 if 'flink' in x.lower() else 0)
df.flink.value_counts()

0    732
1     10
Name: flink, dtype: int64

In [37]:
#mongo db
df['mongo']  = df['Job Description'].apply(lambda x: 1 if 'mongo' in x.lower() else 0)
df.mongo.value_counts()

0    705
1     37
Name: mongo, dtype: int64

In [38]:
#google analytics
df['google_an']  = df['Job Description'].apply(lambda x: 1 if 'google analytics' in x.lower() else 0)
df.google_an.value_counts()

0    728
1     14
Name: google_an, dtype: int64

In [39]:
#drop columns which are too small/zero requirements
df = df.drop("selenium",axis=1)
df = df.drop("R-studio",axis=1)
df = df.drop("julia",axis=1)

In [40]:
df.head(5)

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,...,keras,pytorch,scikit,tensor,hadoop,tableau,bi,flink,mongo,google_an
0,0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,...,0,0,0,0,0,1,1,0,0,0
1,1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,...,0,0,0,0,0,0,0,0,0,0
2,2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,...,0,0,0,0,0,0,0,0,0,0
3,3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,...,0,0,0,0,0,0,0,0,0,0
4,4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,...,0,0,0,0,0,0,0,0,0,0


In [41]:
def seniority(title):
    if 's.sr' in title.lower() or 's. sr' in title.lower() or 's sr' in title.lower() or 'ssr' in title.lower() or 's. senior' in title.lower() or 'semi or' in title.lower() or 'middle' in title.lower() or 'mid' in title.lower():
        return 'ssr'
    elif 'jr' in title.lower() or 'junior' in title.lower():
        return 'jr'
    elif 'sr' in title.lower() or 'senior' in title.lower() or 'lead' in title.lower() or 'principal' in title.lower():
        return 'sr'
    else:
        return 'na'


In [42]:
df['seniority_by_title'] = df['Job Title'].apply(seniority)
df.seniority_by_title.value_counts()

na    519
sr    220
jr      3
Name: seniority_by_title, dtype: int64

In [54]:
df["Job Description"][0]

'Data Scientist\nLocation: Albuquerque, NM\nEducation Required: Bachelor’s degree required, preferably in math, engineering, business, or the sciences.\nSkills Required:\nBachelor’s Degree in relevant field, e.g., math, data analysis, database, computer science, Artificial Intelligence (AI); three years’ experience credit for Master’s degree; five years’ experience credit for a Ph.D\nApplicant should be proficient in the use of Power BI, Tableau, Python, MATLAB, Microsoft Word, PowerPoint, Excel, and working knowledge of MS Access, LMS, SAS, data visualization tools, and have a strong algorithmic aptitude\nExcellent verbal and written communication skills, and quantitative analytical skills are required\nApplicant must be able to work in a team environment\nU.S. citizenship and ability to obtain a DoD Secret Clearance required\nResponsibilities: The applicant will be responsible for formulating analytical solutions to complex data problems; creating data analytic models to improve data

In [45]:
def education(degree): #This function looks for the education degree - Maters or PhD
    if 'master' in degree.lower() or 'masters' in degree.lower() or "master's" in degree.lower():
        return 'M'
    elif 'phd' in degree.lower() or 'ph.d' in degree.lower():
        return 'P'
    else:
        return 'na'

In [46]:
df['Degree'] = df['Job Description'].apply(education)
df.Degree.value_counts()

na    383
M     252
P     107
Name: Degree, dtype: int64

In [48]:
#Export the dataset into another CSV file
df_out=df
df_out.to_csv('data_cleaned.csv',index=False)